In [ ]:
import bw2data as bd
import bw2calc as bc
import bw2analyzer as ba
import bw2io as bi

In [ ]:
if "ecoinvent-3.9-cutoff" not in bd.projects:
    bi.restore_project_directory("/srv/data/projects/ecoinvent-3.9-cutoff.tar.gz")

In [ ]:
bd.projects.set_current("ecoinvent-3.9-cutoff")

In [ ]:
ht = (
    'EF v3.1 no LT',
    'human toxicity: carcinogenic no LT',
    'comparative toxic unit for human (CTUh) no LT'
)

In [ ]:
act = bd.get_activity(database="ecoinvent-3.9-cutoff", name="market for wafer, fabricated, for integrated circuit")

In [ ]:
lca = bc.LCA({act: 1}, ht, use_distributions=True)
lca.lci()
lca.lcia()

In [ ]:
while lca.score >= 0:
    next(lca)

In [ ]:
lca.to_dataframe(matrix_label="characterized_inventory", cutoff=10)

Look up exchange value for first row in inventory matrix

In [ ]:
row = 1027
col = 12309

In [ ]:
lca.inventory[row, col]

Negative value from biosphere or technosphere?

In [ ]:
lca.biosphere_matrix[row, col]

Get information on the uncertainty distribution of this point.

In [ ]:
mask = (
    (lca.biosphere_mm.input_row_col_indices()['row'] == row) *
    (lca.biosphere_mm.input_row_col_indices()['col'] == col)
)

This is [normally distributed](https://stats-arrays.readthedocs.io/en/latest/#mapping-parameter-array-columns-to-uncertainty-distributions), so can be positive or negative. Look how big $\sigma$ is versus $\mu$.

In [ ]:
lca.biosphere_mm.input_uncertainties()[mask]

How does sugarcane get into our wafer production?

In [ ]:
ba.print_recursive_calculation(
    activity=act,
    lcia_method=ht,
    max_level=8,
    cutoff=0.1,
    use_matrix_values=True,
    _lca_obj=lca,
    _total_score=float(lca.score)
)